In [1]:
import ipdb
import os
import csv
import json
import torch
import numpy as np
from argparse import ArgumentParser
import mlearn.modeling.multitask as mtl
from mlearn.utils.metrics import Metrics
from mlearn.data.batching import TorchtextExtractor
from mlearn.data.clean import Cleaner, Preprocessors
from mlearn.utils.train import run_mtl_model, train_mtl_model
from torchtext.data import TabularDataset, Field, LabelField, BucketIterator

In [2]:
# Data inputs and outputs
main = 'davidson'
auxi = ['waseem']
datadir = '../data/json/'
results = '../results/'
save_model = '../results/'

# Cleaning and metrics
cleaners = ['lower', 'username', 'url']
metrics = ['f1-score', 'precision', 'recall', 'accuracy']
display_metric = stop_metric = 'f1-score'
dev_metrics = Metrics(metrics, display_metric, stop_metric)
train_metrics = Metrics(metrics, display_metric, stop_metric)

# Experiment
experiment = 'word'
tokenizer = 'bpe'
seed = 42

# Modelling
# All models
model = 'mlp'
patience = 1
encoding = 'onehot'
loss = 'nlll'
optimizer = 'adam'
shuffle = True
gpu = False
batch_first = True
clip = 1.0

# LSTM
layers = 1

# CNN
window_sizes = "2,3,4"
filters = 128

# Hyper Parameters
embedding = 64
hidden = ["64,64"]
shared = 64
epochs = 10
batch_size = 64
learning_rate = 0.02
dropout = 0.2
nonlinearity = 'tanh'

# MTL specific
batches_epoch = 50
loss_weights = [1.0, 0.5]

In [3]:
# Initialise random seeds
torch.random.manual_seed(seed)
np.random.seed(seed)

# Set up experiment and cleaner
c = Cleaner(processes = cleaners)
exp = Preprocessors('data/').select_experiment(experiment)
onehot = True if encoding == 'onehot' else False

# Load tokenizers
if tokenizer == 'spacy':
    selected_tok  = c.tokenize
elif tokenizer == 'bpe':
    selected_tok = c.bpe_tokenize
elif tokenizer == 'ekphrasis' and args.experiment == 'word':
    selected_tok = c.ekphrasis_tokenize
    annotate = {'elongated', 'emphasis'}
    flters = [f"<{filtr}>" for filtr in annotate]
    c._load_ekphrasis(annotate, flters)
elif tokenizer == 'ekphrasis' and args.experiment == 'liwc':
    ekphr = c.ekphrasis_tokenize
    annotate = {'elongated', 'emphasis'}
    flters = [f"<{filtr}>" for filtr in annotate]
    c._load_ekphrasis(annotate, flters)

    def liwc_toks(doc):
        tokens = ekphr(doc)
        tokens = exp(tokens)
        return tokens
    selected_tok = liwc_toks
tokenizer = selected_tok

In [4]:
# Set up fields
text = Field(tokenize = tokenizer, lower = True, batch_first = True)
label = LabelField()
fields = {'text': ('text', text), 'label': ('label', label)}  # Because we load from json we just need this.

# Load main task training data
if main == 'davidson':
    train, dev, test = TabularDataset.splits(datadir, train = 'davidson_binary_train.json',
                                             validation = 'davidson_binary_dev.json',
                                             test = 'davidson_binary_test.json',
                                             format = 'json', skip_header = True, fields = fields)
text.build_vocab(train)
label.build_vocab(train)
main = {'train': train, 'dev': dev, 'test': test, 'text': text, 'labels': label, 'name': main}

# Load aux tasks
auxillary = []
for i, aux in enumerate(auxi):
    # Set up fields
    text = Field(tokenize = tokenizer, lower = True, batch_first = True)
    label = LabelField()
    fields = {'text': ('text', text), 'label': ('label', label)}  # Because we load from json we just need this.

    if aux == 'davidson':
        train, dev, test = TabularDataset.splits(datadir, train = 'davidson_binary_train.json',
                                                 validation = 'davidson_binary_dev.json',
                                                 test = 'davidson_binary_test.json',
                                                 format = 'json', skip_header = True, fields = fields)
    elif aux == 'hoover':
        train, dev, test = TabularDataset.splits(datadir, train = 'hoover_train.json',
                                                 validation = 'hoover_dev.json',
                                                 test = 'hoover_test.json',
                                                 format = 'json', skip_header = True, fields = fields)
    elif aux == 'oraby_factfeel':
        train, dev, test = TabularDataset.splits(datadir, train = 'oraby_fact_feel_train.json',
                                                 validation = 'oraby_fact_feel_dev.json',
                                                 test = 'oraby_fact_feel_test.json',
                                                 format = 'json', skip_header = True, fields = fields)
    elif aux == 'oraby_sarcasm':
        train, dev, test = TabularDataset.splits(datadir, train = 'oraby_sarcasm_train.json',
                                                 validation = 'oraby_sarcasm_dev.json',
                                                 test = 'oraby_sarcasm_test.json',
                                                 format = 'json', skip_header = True, fields = fields)
    elif aux == 'waseem':
        train, dev, test = TabularDataset.splits(datadir, train = 'waseem_train.json',
                                                 validation = 'waseem_dev.json',
                                                 test = 'waseem_test.json',
                                                 format = 'json', skip_header = True, fields = fields)
    elif aux == 'waseem_hovy':
        train, dev, test = TabularDataset.splits(datadir, train = 'waseem_hovy_train.json',
                                                 validation = 'waseem_hovy_dev.json',
                                                 test = 'waseem_hovy_test.json',
                                                 format = 'json', skip_header = True, fields = fields)
    text.build_vocab(train)
    label.build_vocab(train)
    auxillary.append({'train': train, 'dev': dev, 'test': test, 'text': text, 'labels': label, 'name': aux, 'task_id': i + 1})
    if len(auxillary) == len(auxi): break

In [5]:
# Hyper parameters
dropout = dropout
nonlinearity = nonlinearity
learning_rate = learning_rate
epochs = epochs
batch_size = batch_size
batch_count = batches_epoch
loss_weights = loss_weights

params = dict(shared_dim = shared,
              batch_first = True,
              hidden_dims = [int(hidden) for hidden in hidden[0].split(',')],
              input_dims = [len(main['text'].vocab.stoi)] + [len(aux['text'].vocab.stoi) for aux in auxillary],
              output_dims = [len(main['labels'].vocab.stoi)] + [len(aux['labels'].vocab.stoi) for aux in auxillary],
              )
print(params)

{'shared_dim': 64, 'batch_first': True, 'hidden_dims': [64, 64], 'input_dims': [18176, 9826], 'output_dims': [2, 4]}


In [6]:
if not onehot:
    params.update({'embedding_dims': embedding})
if model == 'lstm':
    params.update({'no_layers': layers})
    model = mtl.OnehotLSTMClassifier if onehot else mtl.EmbeddingLSTMClassifier
else:
    params.update({'non-linearity': nonlinearity})
    model = mtl.OnehotMLPClassifier if onehot else mtl.EmbeddingMLPClassifier

    if model == 'cnn':
        params.update({'window_sizes': [int(win) for win in window_sizes[0].split(',')],
                       'num_filters': filters})
        model = mtl.OnehotCNNClassifier if onehot else mtl.EmbeddingCNNClassifier
    elif model == 'mlp':
        model = mtl.OnehotMLPClassifier if onehot else mtl.EmbeddingMLPClassifier

model = model(**params)
print(model)

OnehotMLPClassifier(
  (all_parameters): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 64x18176]
      (1): Parameter containing: [torch.FloatTensor of size 64]
      (2): Parameter containing: [torch.FloatTensor of size 64x9826]
      (3): Parameter containing: [torch.FloatTensor of size 64]
      (4): Parameter containing: [torch.FloatTensor of size 64x64]
      (5): Parameter containing: [torch.FloatTensor of size 64]
      (6): Parameter containing: [torch.FloatTensor of size 64x64]
      (7): Parameter containing: [torch.FloatTensor of size 64]
      (8): Parameter containing: [torch.FloatTensor of size 64x64]
      (9): Parameter containing: [torch.FloatTensor of size 64]
      (10): Parameter containing: [torch.FloatTensor of size 2x64]
      (11): Parameter containing: [torch.FloatTensor of size 2]
      (12): Parameter containing: [torch.FloatTensor of size 4x64]
      (13): Parameter containing: [torch.FloatTensor of size 4]
  )
  (shared): Linear

In [7]:
# Info about losses: https://bit.ly/3irxvYK
if loss == 'nlll':
    loss = torch.nn.NLLLoss()
elif loss == 'crossentropy':
    loss = torch.nn.CrossEntropyLoss()

# Set optimizer
if optimizer == 'adam':
    optimizer = torch.optim.Adam(model.parameters(), learning_rate)
elif optimizer == 'sgd':
    optimizer = torch.optim.SGD(model.parameters(), learning_rate)
elif optimizer == 'asgd':
    optimizer = torch.optim.ASGD(model.parameters(), learning_rate)
elif optimizer == 'adamw':
    optimizer = torch.optim.AdamW(model.parameters(), learning_rate)

In [8]:
# Batch data
batchers = []
test_batchers = []
if onehot:
    print("onehot")
    train_buckets = BucketIterator(dataset = main['train'], batch_size = batch_size, sort_key = lambda x: len(x))
    train = TorchtextExtractor('text', 'label', main['name'], train_buckets, len(main['text'].vocab.stoi))
    batchers.append(train)

    dev_buckets = BucketIterator(dataset = main['dev'], batch_size = 64, sort_key = lambda x: len(x))
    dev = TorchtextExtractor('text', 'label', main['name'], dev_buckets, len(main['text'].vocab.stoi))

    test_buckets = BucketIterator(dataset = main['test'], batch_size = 64, sort_key = lambda x: len(x))
    main_test = TorchtextExtractor('text', 'label', main['name'], test_buckets, len(main['text'].vocab.stoi))

    for aux in auxillary:
        train_buckets = BucketIterator(dataset = aux['train'], batch_size = batch_size, sort_key = lambda x: len(x))
        train = TorchtextExtractor('text', 'label', aux['name'], train_buckets, len(aux['text'].vocab.stoi))
        batchers.append(train)

        test_buckets = BucketIterator(dataset = aux['test'], batch_size = 64, sort_key = lambda x: len(x))
        test = TorchtextExtractor('text', 'label', aux['name'], test_buckets, len(aux['text'].vocab.stoi))
        test_batchers.append(test)
            
else:
    print("embedding")
    train_buckets = BucketIterator(dataset = main['train'], batch_size = batch_size, sort_key = lambda x: len(x))
    main_train = TorchtextExtractor('text', 'label', main['name'], train_buckets)
    batchers.append(main_train)

    dev_buckets = BucketIterator(dataset = main['dev'], batch_size = 64, sort_key = lambda x: len(x))
    dev = TorchtextExtractor('text', 'label', main['name'], dev_buckets)

    test_buckets = BucketIterator(dataset = main['test'], batch_size = 64, sort_key = lambda x: len(x))
    main_test = TorchtextExtractor('text', 'label', main['name'], test_buckets)

    for aux in auxillary:
        train_buckets = BucketIterator(dataset = aux['train'], batch_size = batch_size, sort_key = lambda x: len(x))
        train = TorchtextExtractor('text', 'label', aux['name'], train_buckets)
        batchers.append(train)

        test_buckets = BucketIterator(dataset = aux['test'], batch_size = 64, sort_key = lambda x: len(x))
        test = TorchtextExtractor('text', 'label', aux['name'], test_buckets)
        test_batchers.append(test)
next(iter(batchers[0]))[0].shape

onehot


torch.Size([64, 58, 18176])

In [9]:
next(iter(train.data))


[torchtext.data.batch.Batch of size 64]
	[.text]:[torch.LongTensor of size 64x41]
	[.label]:[torch.LongTensor of size 64]

In [10]:
batch_writing = dict(batch_writer = csv.writer(open('test', 'w')),
                     model_hdr = ['name'], main_name = 'davidson', hyper_info = [embedding],
                     metric_hdr = ['f1-score'], writer = csv.writer(open('test2', 'w')), data_name = 'davidson')
modelling_vars = dict(model = model, 
                      batchers = batchers, 
                      optimizer = optimizer, 
                      loss = loss,
                      metrics = train_metrics, 
                      batch_size = batch_size, 
                      epochs = epochs, 
                      clip = clip,
                      early_stopping = patience, 
                      save_model = results, 
                      dev = dev, 
                      dev_metrics = dev_metrics,
                      dev_task_id = 0, 
                      batches_per_epoch = batches_epoch, 
                      low = False, 
                      shuffle = False, 
                      dataset_weights = None, 
                      loss_weights = loss_weights, 
                      gpu = False, 
                      hyperopt = False)

In [11]:
run_mtl_model(train = True, **modelling_vars, **batch_writing)

Batch:  64%|██████▍   | 32/50 [00:28<00:12,  1.42it/s, batch_loss=0.0354, epoch_loss=0.0321, task=0, task_score=0.0314]


Batch: 100%|██████████| 50/50 [00:39<00:00,  1.86it/s, batch_loss=0.0144, epoch_loss=0.0288, task=1, task_score=0.0561]
                                                                                                                       
Batch:   6%|▌         | 3/50 [00:02<00:24,  1.92it/s, batch_loss=0.0297, epoch_loss=0.0260, task=0, task_score=0.0417]


Batch:  72%|███████▏  | 36/50 [00:19<00:05,  2.54it/s, batch_loss=0.0159, epoch_loss=0.0210, task=1, task_score=0.0313]


Batch: 100%|██████████| 50/50 [00:27<00:00,  1.57it/s, batch_loss=0.0243, epoch_loss=0.0215, task=0, task_score=0.0404]
                                                                                                                       
Batch:  14%|█▍        | 7/50 [00:05<00:35,  1.21it/s, batch_loss=0.0333, epoch_loss=0.0242, task=0, task_score=0.0319]


Batch:  80%|████████  | 40/50 [00:23<00:05,  1.70it/s, batch_loss=0.0163, epoch_loss=0.0202, task=1, task_score=0.0528]


Batch: 100%|██████████| 50/50 [00:27<00:00,  2.57it/s, batch_loss=0.0109, epoch_loss=0.0194, task=1, task_score=0.0668]
                                                                                                                       
Batch:  22%|██▏       | 11/50 [00:06<00:25,  1.55it/s, batch_loss=0.0113, epoch_loss=0.0209, task=1, task_score=0.0403]


Batch:  88%|████████▊ | 44/50 [00:26<00:09,  1.55s/it, batch_loss=0.0091, epoch_loss=0.0199, task=1, task_score=0.1204]


Batch: 100%|██████████| 50/50 [00:29<00:00,  1.04it/s, batch_loss=0.0270, epoch_loss=0.0202, task=0, task_score=0.0492]
                                                                                                                       
Evaluating model: 100%|██████████| 39/39 [00:22<00:00,  1.56it/s]
                                                                                                              

Early stopping: Terminate
Loading weights from epoch 2


In [13]:
print(dev_metrics.scores)

{'f1-score': [0.019746062875596253, 0.020510140338038184, 0.020493812060904146, 0.019956901033051994], 'precision': [0.015730650075595327, 0.016424099417566985, 0.016517081511881273, 0.01612070826570358], 'recall': [0.026915195684188285, 0.027974030623037244, 0.027859387924381994, 0.026960222760604544], 'accuracy': [0.4412595882115462, 0.4578118691966088, 0.465482438433589, 0.46669358094469116], 'loss': [0.02935877652913272, 0.02839940134247898, 0.030988091945070677, 0.0261436183461269]}


In [14]:
# Do tests
main_task_eval = dict(model = model,
                      batchers = main_test,
                      loss = loss,
                      metrics = Metrics(metrics, display, stop_metric),
                      gpu = gpu,
                      mtl = 0,
                      store = False,
                      data = None,
                      writer = csv.writer(open('test3', 'w')),
                      hyper_info = [embedding],
                      model_hdr = ['name'],
                      metric_hdr = ['f1-score'],
                      main_name = 'davidson',
                      data_name = 'davidson'
                      )

run_mtl_model(train = False, **main_task_eval)

In [15]:
for i, aux in enumerate(test_batchers):
    print(i)
    print(auxillary[i]['task_id'])
    aux_metrics = Metrics(metrics, display, stop_metric)
    aux_dict = dict(model = model,
                    batchers = aux,
                    metrics = aux_metrics,
                    gpu = gpu,
                    mtl = auxillary[i]['task_id'],
                    store = False,
                    data = None,
                    data_name = auxillary[i]['name'],
                    main_name = 'davidson',
                    writer = csv.writer(open('test4', 'w')),
                    loss = loss,
                    hyper_info = [embedding],
                    model_hdr = ['name'],
                    metric_hdr = ['f1-score'],
                    )
    run_mtl_model(train = False, **aux_dict)
    aux_metrics = {f"{auxillary[i]['name']}_{m}": key for m, key in aux_metrics.scores.items()}

Evaluating model:   0%|          | 0/11 [00:00<?, ?it/s]

0
1


In [ ]:
auxillary

In [ ]:
test_batchers